```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago
```

# Solución: ¿Qué tan memorable es un video?

Este notebook les permitirá tener a disposición todos los recursos que se vieron en durante la materia. Todos los fragmentos de código se descargaran utilizando la sección preparación del ambiente.

## Preparación del ambiente

### Sets de datos

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/ground_truth.csv --directory-prefix ./Data/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/audio_vectors.csv --directory-prefix ./Data/Features/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/c3d_vectors.csv --directory-prefix ./Data/Features/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/caption_features.csv --directory-prefix ./Data/Features/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/colors_features.csv --directory-prefix ./Data/Features/

### Librerías de codigo que se utilizan durante el curso

#### Audio

In [ ]:
!wget -N https://storage.googleapis.com/audioset/yamnet.h5 --directory-prefix ./Models/yamnet/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/Models/yamnet/yamnet_class_map.csv --directory-prefix ./Models/yamnet/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/Utils/audio_plotter.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/yamnet/yamnet.py --directory-prefix ./yamnet/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/yamnet/params.py --directory-prefix ./yamnet/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/yamnet/features_lib.py --directory-prefix ./yamnet/

In [ ]:
!pip install librosa
!pip install moviepy
!pip install soundfile
!pip install azure-cognitiveservices-speech

#### NLP

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/ClassificationDataset.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextNormalizer.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/PadSequenceTransformer.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/Word2VecVectorizer.py --directory-prefix ./Utils/
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model.bin --directory-prefix ./Models/Word2Vec

In [ ]:
!pip install transformers
!pip install unidecode
!python -m spacy download es_core_news_sm

#### Vision

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Vision/Utils/Cognitive.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Vision/Utils/Plotting.py --directory-prefix ./Utils/

In [ ]:
!pip install tensorflow-datasets
!pip install azure-cognitiveservices-vision-computervision
!pip install matplotlib
!pip install skimage

### Imports

In [ ]:
import pandas as pd
import numpy as np
import librosa
import os
import soundfile as sf
import torch
import transformers
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D
import tensorflow_datasets as tfds

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from yamnet.yamnet import yamnet_frames_model
import yamnet.yamnet as yamnet_model
from yamnet.params import Params

from Utils.audio_plotter import plot_audio_embeddings
from Utils.ClassificationDataset import ClassificationDataset
from Utils.TextNormalizer import TweetTextNormalizer
from Utils.Word2VecVectorizer import Word2VecVectorizer
from Utils.PadSequenceTransformer import PadSequenceTransformer

import azure.cognitiveservices.speech as speechsdk
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

## Solución

Cuentan con la anotaciones para el problema de memorabilidad. Dependiendo de su elección pueden optar por resolver un problema de clasificación o un problema de regresión. Esto lo veran refrejado en:
 - **memorability_score:** Representa el puntaje de memorabilidad de la secuencia en particular. Valores altos son mejores.
 - **memorable:** Variable categórica que representa si un video es memorable o no. Un video con Score superior a 0.5 es marcado como memorable (1), sino es marcado como no memorable (0)

In [ ]:
labels = pd.read_csv('Data/ground_truth.csv')

In [ ]:
labels.head(2)

,movie_name,start(sec),end(sec),sequence_name,Neutral (1)_Typical (0),nb_annotations,memorability_score,memorabable
0,127 hours,2000,2010,127_hours_2000_2010_1,0,5,1.0,1
1,127 hours,2182,2192,127_hours_2182_2192_5,1,8,0.0,0


### Para aquellos que utilicen las features preprocesadas

Las siguientes variables contienen la información preprocesada:

In [ ]:
captions = pd.read_csv('Data/Features/caption_features.csv')
colors = pd.read_csv('Data/Features/colors_features.csv')
audio_embeddings = pd.read_csv('Data/Features/audio_vectors.csv')
video_embeddings = pd.read_csv('Data/Features/c3d_vectors.csv')

In [ ]:
# completar solución

### Para aquellos que utilicen los datos de origen

Dado que el tamaño de los datos es bastante grande, pueden optar por que set de datos utilizar:

#### Para utilizar los cuadros (frames)

In [ ]:
!wget -N https://santiagxf.blob.core.windows.net/public/Memorability/frames.tar.gz --directory-prefix ./Data/Raw/
!tar zxvf ./Data/Raw/frames.tar.gz --directory ./Data/Raw/

In [8]:
#En el directorio Data/Raw/frames encontraran todos los frames extraidos de cada una de las secuencias. Los frames están disponibles cada 2 segundos por lo cual disponen de 5 frames por cada secuencia.

#### Para utilizar los audios

In [ ]:
!wget -N https://santiagxf.blob.core.windows.net/public/Memorability/audios.tar.gz --directory-prefix ./Data/Raw/
!tar zxvf ./Data/Raw/audios.tar.gz --directory ./Data/Raw/

In [6]:
#En el directorio Data/Raw/audios encontraran todos los audios correspondientes a cada una de las secuencias de los videos. Los audios están en formato wav, que si bien ocupan mayor espacio, son más sencillos de procesar.

#### Para utilizar los videos

In [ ]:
!wget -N https://santiagxf.blob.core.windows.net/public/Memorability/sources.tar.gz --directory-prefix ./Data/Raw/
!tar zxvf ./Data/Raw/sources.tar.gz --directory ./Data/Raw/

In [ ]:
#En el directorio Data/Raw/sources encontraran todas las secuencias utilizadas en este desafío. El archivo 'Data/ground_truth.csv' contine los nombres de las peliculas a las que corresponde cada fragmento en caso que quieran revisarlos.

In [ ]:
# completar solución